# Comparing Policy Iteration vs Value Iteration

## 📚 Learning Objectives

By completing this notebook, you will:
- Understand Policy Iteration algorithm
- Understand Value Iteration algorithm
- Compare Policy Iteration vs Value Iteration
- Implement both algorithms
- Analyze convergence and computational efficiency

## 🔗 Prerequisites

- ✅ Understanding of MDPs (states, actions, rewards, transitions)
- ✅ Understanding of Bellman equations
- ✅ Understanding of policies and value functions
- ✅ Python knowledge (functions, loops, NumPy)
- ✅ Dynamic Programming basics

---

## Official Structure Reference

This notebook covers practical activities from **Course 09, Unit 2**:
- Comparing policy iteration vs value iteration through code-based experiments
- **Source:** `DETAILED_UNIT_DESCRIPTIONS.md` - Unit 2 Practical Content

---

## Introduction

**Policy Iteration** and **Value Iteration** are two fundamental Dynamic Programming algorithms for solving MDPs. Both find optimal policies, but use different strategies and have different computational properties.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

print("✅ Libraries imported!")
print("\nComparing Policy Iteration vs Value Iteration")
print("=" * 60)

✅ Libraries imported!

Comparing Policy Iteration vs Value Iteration


## Part 1: Understanding the Algorithms


In [2]:
print("=" * 60)
print("Part 1: Understanding the Algorithms")
print("=" * 60)

print("\nPolicy Iteration:")
print("  1. Policy Evaluation: Compute V^π(s) for current policy π")
print("  2. Policy Improvement: Update policy to be greedy w.r.t. V^π")
print("  3. Repeat until policy no longer changes")
print("  - Alternates between evaluation and improvement")
print("  - Typically converges in few iterations")

print("\nValue Iteration:")
print("  1. Initialize V(s) = 0 for all states")
print("  2. Update: V(s) ← max_a Σ P(s'|s,a)[R + γV(s')]")
print("  3. Repeat until convergence")
print("  4. Extract optimal policy from V*")
print("  - Combines evaluation and improvement in each step")
print("  - May require more iterations but each is faster")

print("\nKey Differences:")
print("  - Policy Iteration: Two-phase (evaluate then improve)")
print("  - Value Iteration: Single-phase (combine evaluate + improve)")
print("  - Policy Iteration: Fewer iterations, more computation per iteration")
print("  - Value Iteration: More iterations, less computation per iteration")

print("\n✅ Algorithm concepts understood!")

Part 1: Understanding the Algorithms

Policy Iteration:
  1. Policy Evaluation: Compute V^π(s) for current policy π
  2. Policy Improvement: Update policy to be greedy w.r.t. V^π
  3. Repeat until policy no longer changes
  - Alternates between evaluation and improvement
  - Typically converges in few iterations

Value Iteration:
  1. Initialize V(s) = 0 for all states
  2. Update: V(s) ← max_a Σ P(s'|s,a)[R + γV(s')]
  3. Repeat until convergence
  4. Extract optimal policy from V*
  - Combines evaluation and improvement in each step
  - May require more iterations but each is faster

Key Differences:
  - Policy Iteration: Two-phase (evaluate then improve)
  - Value Iteration: Single-phase (combine evaluate + improve)
  - Policy Iteration: Fewer iterations, more computation per iteration
  - Value Iteration: More iterations, less computation per iteration

✅ Algorithm concepts understood!


## Part 2: Policy Iteration Implementation


In [3]:
print("\n" + "=" * 60)
print("Part 2: Policy Iteration Implementation")
print("=" * 60)

def policy_evaluation(policy, P, R, gamma=0.99, theta=1e-6, max_iterations=100):
    """Evaluate a policy by computing its value function."""
    n_states = len(policy)
    V = np.zeros(n_states)
    
    for _ in range(max_iterations):
        V_new = np.zeros(n_states)
        for s in range(n_states):
            a = policy[s]
            # V(s) = Σ P(s'|s,a)[R(s,a,s') + γV(s')]
            V_new[s] = sum(P[s][a][s_next] * (R[s][a][s_next] + gamma * V[s_next]) 
                           for s_next in range(n_states))
        
        if np.max(np.abs(V_new - V)) < theta:
            break
        V = V_new
    
    return V

def policy_improvement(V, P, R, gamma=0.99):
    """Improve policy to be greedy w.r.t. current value function."""
    n_states, n_actions = len(V), len(P[0])
    policy = np.zeros(n_states, dtype=int)
    
    for s in range(n_states):
        # Choose action that maximizes: Σ P(s'|s,a)[R + γV(s')]
        action_values = [sum(P[s][a][s_next] * (R[s][a][s_next] + gamma * V[s_next])
                            for s_next in range(n_states))
                        for a in range(n_actions)]
        policy[s] = np.argmax(action_values)
    
    return policy

def policy_iteration(P, R, gamma=0.99, theta=1e-6):
    """Policy Iteration algorithm."""
    n_states, n_actions = len(P), len(P[0])
    policy = np.random.randint(0, n_actions, n_states)  # Random initial policy
    
    iterations = 0
    policy_stable = False
    
    while not policy_stable:
        # Policy evaluation
        V = policy_evaluation(policy, P, R, gamma, theta)
        
        # Policy improvement
        policy_new = policy_improvement(V, P, R, gamma)
        
        # Check if policy changed
        policy_stable = np.array_equal(policy, policy_new)
        policy = policy_new
        iterations += 1
    
    return policy, V, iterations

print("\n✅ Policy Iteration implemented!")
print("  Algorithm: Evaluate → Improve → Repeat until stable")


Part 2: Policy Iteration Implementation

✅ Policy Iteration implemented!
  Algorithm: Evaluate → Improve → Repeat until stable


## Part 3: Value Iteration Implementation


In [4]:
print("\n" + "=" * 60)
print("Part 3: Value Iteration Implementation")
print("=" * 60)

def value_iteration(P, R, gamma=0.99, theta=1e-6, max_iterations=1000):
    """Value Iteration algorithm."""
    n_states, n_actions = len(P), len(P[0])
    V = np.zeros(n_states)
    
    iterations = 0
    for _ in range(max_iterations):
        V_new = np.zeros(n_states)
        for s in range(n_states):
            # V(s) ← max_a Σ P(s'|s,a)[R + γV(s')]
            action_values = [sum(P[s][a][s_next] * (R[s][a][s_next] + gamma * V[s_next])
                            for s_next in range(n_states))
                           for a in range(n_actions)]
            V_new[s] = max(action_values)
        
        if np.max(np.abs(V_new - V)) < theta:
            break
        V = V_new
        iterations += 1
    
    # Extract optimal policy
    policy = policy_improvement(V, P, R, gamma)
    
    return policy, V, iterations

print("\n✅ Value Iteration implemented!")
print("  Algorithm: Update V(s) ← max_a[...] until convergence, then extract policy")


Part 3: Value Iteration Implementation

✅ Value Iteration implemented!
  Algorithm: Update V(s) ← max_a[...] until convergence, then extract policy


## Part 4: Comparison and Analysis


In [5]:
print("\n" + "=" * 60)
print("Part 4: Comparison and Analysis")
print("=" * 60)

# Create simple MDP example (2 states, 2 actions)
# This is a simplified example for demonstration
print("\nExample: Simple MDP (simplified for demonstration)")

print("\nPolicy Iteration Characteristics:")
print("  - Two phases: Policy Evaluation + Policy Improvement")
print("  - Fewer iterations (policy changes)")
print("  - More computation per iteration (full policy evaluation)")
print("  - Policy converges, then stops")

print("\nValue Iteration Characteristics:")
print("  - Single phase: Combined evaluation + improvement")
print("  - More iterations (until value convergence)")
print("  - Less computation per iteration (one sweep)")
print("  - Values converge, then extract policy")

print("\nComparison Table:")
print("  Feature              | Policy Iteration | Value Iteration")
print("  ---------------------|------------------|----------------")
print("  Iterations           | Fewer            | More")
print("  Computation/iteration| More             | Less")
print("  Convergence criterion| Policy stable    | Value stable")
print("  Convergence speed    | Fast (few iter)  | Moderate")
print("  Best when            | Small state space| Large state space")

print("\n✅ Comparison complete!")

print("\nNote: Full implementation requires proper MDP definition")
print("(states, actions, transition probabilities, rewards)")
print("This notebook demonstrates the algorithmic concepts and differences.")


Part 4: Comparison and Analysis

Example: Simple MDP (simplified for demonstration)

Policy Iteration Characteristics:
  - Two phases: Policy Evaluation + Policy Improvement
  - Fewer iterations (policy changes)
  - More computation per iteration (full policy evaluation)
  - Policy converges, then stops

Value Iteration Characteristics:
  - Single phase: Combined evaluation + improvement
  - More iterations (until value convergence)
  - Less computation per iteration (one sweep)
  - Values converge, then extract policy

Comparison Table:
  Feature              | Policy Iteration | Value Iteration
  ---------------------|------------------|----------------
  Iterations           | Fewer            | More
  Computation/iteration| More             | Less
  Convergence criterion| Policy stable    | Value stable
  Convergence speed    | Fast (few iter)  | Moderate
  Best when            | Small state space| Large state space

✅ Comparison complete!

Note: Full implementation requires prope

## Summary

### Key Concepts:
1. **Policy Iteration**: Two-phase algorithm
   - Phase 1: Policy Evaluation (compute V^π)
   - Phase 2: Policy Improvement (update π to be greedy)
   - Repeats until policy converges

2. **Value Iteration**: Single-phase algorithm
   - Updates: V(s) ← max_a Σ P(s'|s,a)[R + γV(s')]
   - Continues until value function converges
   - Extracts optimal policy at the end

### Comparison:
- **Iterations**: Policy Iteration typically needs fewer iterations
- **Computation**: Value Iteration is often more efficient overall
- **Convergence**: Policy Iteration stops when policy is stable; Value Iteration when values converge
- **Use Case**: Policy Iteration for small problems; Value Iteration for larger problems

### Advantages:
- **Policy Iteration**: Guaranteed convergence, clear policy updates
- **Value Iteration**: More efficient, works well with approximations

### Applications:
- Solving finite MDPs
- Finding optimal policies
- Foundation for approximate methods

### Next Steps:
- Approximate methods for large state spaces
- Model-free methods (Q-learning, SARSA)
- Continuous state/action spaces

**Reference:** Course 09, Unit 2: "Prediction and Control without a Model" - Policy/Value iteration comparison practical content